In [3]:
import cv2
import math
import numpy as np
import imutils
from Line import Line
from Square import Square
from Board import Board

debug =  True


class board_Recognition:
	'''
	This class handles the initialization of the board. It analyzes
	the empty board finding its border, lines, corners, squares...
	
    
	def __init__(self, camera):

		self.cam = camera
    '''

	def initialize_Board(self):

		corners = []

		# retake picture until board is initialized properly
		while len(corners) < 81:

			image = cv2.imread(r'C:\Users\mayuresh\Desktop\ESE205-CVChess-master\capture1.jpg',1)

			# Binarize the photo
			adaptiveThresh,img = self.clean_Image(image)

			# Black out all pixels outside the border of the chessboard
			mask = self.initialize_mask(adaptiveThresh,img)

			# Find edges
			edges,colorEdges = self.findEdges(mask)

			# Find lines
			horizontal, vertical = self.findLines(edges,colorEdges)

			# Find corners
			corners = self.findCorners(horizontal, vertical, colorEdges)

		# Find squares
		squares = self.findSquares(corners, img)
		# create Board
		board = Board(squares)

		return board

	def clean_Image(self,image):
		'''
		Resizes and converts the photo to black and white for simpler analysis
		'''
		# resize image
		img = imutils.resize(image, width=400, height = 400)
		
		# Convert to grayscale
		gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

		# Setting all pixels above the threshold value to white and those below to black
		# Adaptive thresholding is used to combat differences of illumination in the picture
		adaptiveThresh = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 125, 1)
        #change to debug
		if(debug):
			# Show thresholded image
			cv2.imshow("Adaptive Thresholding", adaptiveThresh)
			cv2.waitKey(0)
			cv2.destroyAllWindows()

		return adaptiveThresh,img

	def initialize_mask(self, adaptiveThresh,img):
		'''
		Finds border of chessboard and blacks out all unneeded pixels
		'''

		# Find contours (closed polygons)
		contours, hierarchy = cv2.findContours(adaptiveThresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

		# Create copy of original image
		imgContours = img.copy()

		for c in range(len(contours)):
			# Area
			area = cv2.contourArea(contours[c])
			# Perimenter
			perimeter = cv2.arcLength(contours[c], True)
				# Filtering the chessboard edge / Error handling as some contours are so small so as to give zero division
				#For test values are 70-40, for Board values are 80 - 75 - will need to recalibrate if change
				#the largest square is always the largest ratio
			if c ==0:
				Lratio = 0
			if perimeter > 0:
				ratio = area / perimeter
				if ratio > Lratio:
					largest=contours[c]
					Lratio = ratio
					Lperimeter=perimeter
					Larea = area
			else:
					pass

		# Draw contours
		cv2.drawContours(imgContours, [largest], -1, (0,0,0), 1)
		if debug:
			# Show image with contours drawn
			cv2.imshow("Chess Boarder",imgContours)
			cv2.waitKey(0)
			cv2.destroyAllWindows()

		# Epsilon parameter needed to fit contour to polygon
		epsilon = 0.1 * Lperimeter
		# Approximates a polygon from chessboard edge
		chessboardEdge = cv2.approxPolyDP(largest, epsilon, True)

		# Create new all black image
		mask = np.zeros((img.shape[0], img.shape[1]), 'uint8')*125
		# Copy the chessboard edges as a filled white polygon size of chessboard edge
		cv2.fillConvexPoly(mask, chessboardEdge, 255, 1)
		# Assign all pixels that are white (i.e the polygon, i.e. the chessboard)
		extracted = np.zeros_like(img)
		extracted[mask == 255] = img[mask == 255]
		# remove strip around edge
		extracted[np.where((extracted == [125, 125, 125]).all(axis=2))] = [0, 0, 20]

		if debug:
			# Show image with mask drawn
			cv2.imshow("mask",extracted)
			cv2.waitKey(0)
			cv2.destroyAllWindows()
		return extracted

	def findEdges(self, image):
		'''
		Finds edges in the image. Edges later used to find lines and so on
		'''
	
		# Find edges
		edges = cv2.Canny(image, 100, 200, None, 3)
		if debug:
			#Show image with edges drawn
			cv2.imshow("Canny", edges)
			cv2.waitKey(0)
			cv2.destroyAllWindows()

		# Convert edges image to grayscale
		colorEdges = cv2.cvtColor(edges,cv2.COLOR_GRAY2BGR)

		return edges,colorEdges

	def findLines (self, edges, colorEdges):
		'''
		Finds the lines in the photo and sorts into vertical and horizontal
		'''
		
		# Infer lines based on edges
		lines = cv2.HoughLinesP(edges, 1,  np.pi / 180, 100,np.array([]), 100, 80)

		# Draw lines
		a,b,c = lines.shape
		for i in range(a):
			cv2.line(colorEdges, (lines[i][0][0], lines[i][0][1]), (lines[i][0][2], lines[i][0][3]), (0,255,0),2,cv2.LINE_AA)

		if  debug:
			# Show image with lines drawn
			cv2.imshow("Lines",colorEdges)
			cv2.waitKey(0)
			cv2.destroyAllWindows()

		# Create line objects and sort them by orientation (horizontal or vertical)
		horizontal = []
		vertical = []
		for l in range(a):
			[[x1,y1,x2,y2]] = lines[l]
			newLine = Line(x1,x2,y1,y2)
			if newLine.orientation == 'horizontal':
				horizontal.append(newLine)
			else:
				vertical.append(newLine)

		return horizontal, vertical

	def findCorners (self, horizontal, vertical, colorEdges):
		'''
		Finds corners at intersection of horizontal and vertical lines.
		'''

		# Find corners (intersections of lines)
		corners = []
		for v in vertical:
			for h in horizontal:
				s1,s2 = v.find_intersection(h)
				corners.append([s1,s2])

		# remove duplicate corners
		dedupeCorners = []
		for c in corners:
			matchingFlag = False
			for d in dedupeCorners:
				if math.sqrt((d[0]-c[0])*(d[0]-c[0]) + (d[1]-c[1])*(d[1]-c[1])) < 20:
					matchingFlag = True
					break
			if not matchingFlag:
				dedupeCorners.append(c)

		for d in dedupeCorners:
			cv2.circle(colorEdges, (d[0],d[1]), 10, (0,0,255))


		if debug:
			#Show image with corners circled
			cv2.imshow("Corners",colorEdges)
			cv2.waitKey(0)
			cv2.destroyAllWindows()

		return dedupeCorners

	def findSquares(self, corners, colorEdges):
		'''
		Finds the squares of the chessboard 
		'''

		# sort corners by row
		corners.sort(key=lambda x: x[0])
		rows = [[],[],[],[],[],[],[],[],[]]
		r = 0
		for c in range(0, 81):
			if c > 0 and c % 9 == 0:
				r = r + 1

			rows[r].append(corners[c])

		letters = ['a','b','c','d','e','f','g','h']
		numbers = ['1','2','3','4','5','6','7','8']
		Squares = []
		
		# sort corners by column
		for r in rows:
			r.sort(key=lambda y: y[1])
		
		# initialize squares
		for r in range(0,8):
			for c in range (0,8):
				c1 = rows[r][c]
				c2 = rows[r][c + 1]
				c3 = rows[r + 1][c]
				c4 = rows[r + 1][c + 1]

				position = letters[r] + numbers[7-c]
				newSquare = Square(colorEdges,c1,c2,c3,c4,position)
				newSquare.draw(colorEdges,(0,0,255),2)
				newSquare.drawROI(colorEdges,(255,0,0),2)
				newSquare.classify(colorEdges)
				Squares.append(newSquare)



		if debug:
			#Show image with squares and ROI drawn and position labelled
			cv2.imshow("Squares", colorEdges)
			cv2.waitKey(0)
			cv2.destroyAllWindows()

		return Squares


In [37]:
import math
import cv2
import numpy as np
import imutils

In [20]:
import cv2
import numpy as np
import math

debug = True


class Board:
	"""
	Holds all the Square instances and updates changes to board after moves
	"""
	def __init__(self, squares):

		self.squares = squares
		self.boardMatrix = []
		self.promotion = 'q'
		self.promo = False
		self.move = "e2e4"

	def draw(self,image):
		"""
		Draws the board and classifies the squares (draws the square state on the image).
		"""
		for square in self.squares:
			square.draw(image, (0,0,255))
			square.classify(image)

	def assignState(self):
		"""
		Assigns initial setup states to squares and initializes the Board matrix.
		"""
		black = ['r', 'n', 'b','q','k','b','n','r']
		white = ['R','N','B','Q','K','B','N','R']

		for i in range(8):
			self.squares[8*i + 0].state = black[i]
			self.squares[8*i + 1].state = 'p'
			self.squares[8*i + 2].state = '.'
			self.squares[8*i + 3].state = '.'
			self.squares[8*i + 4].state = '.'
			self.squares[8*i + 5].state = '.'
			self.squares[8*i + 6].state = 'P'
			self.squares[8*i + 7].state = white[i]

		for square in self.squares:
			self.boardMatrix.append(square.state)

	def determineChanges(self,previous, current):
		'''
		Determines the change in color values within squares from picture to picture
		to infer piece movement
		'''

		copy = current.copy()
		
		largestSquare = 0
		secondLargestSquare = 0
		largestDist = 0
		secondLargestDist = 0
		stateChange = []

		# check for differences in color between the photos
		for sq in self.squares:
			colorPrevious = sq.roiColor(previous)
			colorCurrent = sq.roiColor(current)

			# distance in bgr values
			sum = 0
			for i in range(0,3):
				sum += (colorCurrent[i] - colorPrevious[i])**2

			distance = math.sqrt(sum)

			if distance > 25:
				stateChange.append(sq)
				
			if distance > largestDist:
				# update squares with largest change in color
				secondLargestSquare = largestSquare
				secondLargestDist = largestDist
				largestDist = distance
				largestSquare = sq

			elif distance > secondLargestDist:
				# update second change in color
				secondLargestDist = distance
				secondLargestSquare = sq


		if  len(stateChange)  == 4:
			
			# if four square have color change in a single move, castling took place
			squareOne = stateChange[0]
			squareTwo = stateChange[1]
			squareThree = stateChange[2]
			squareFour = stateChange[3]

			# check White short side castle
			if squareOne.position == "e1" or squareTwo.position == "e1" or squareThree.position == "e1" or  squareFour.position == "e1":
				if squareOne.position == "f1"  or squareTwo.position == "f1" or squareThree.position == "f1"  or squareFour.position == "f1":
					if squareOne.position == "g1" or squareTwo.position == "g1" or squareThree.position == "g1" or  squareFour.position == "g1":
						if squareOne.position == "h1"  or squareTwo.position == "h1" or squareThree.position == "h1"  or squareFour.position == "h1":
							self.move = "e1g1"
							print(self.move)
							if debug:
								squareOne.draw(copy, (255,0,0), 2)
								squareTwo.draw(copy, (255,0,0), 2)
								squareThree.draw(copy, (255,0,0),2)
								squareFour.draw(copy, (255,0,0), 2)
								cv2.imshow("previous",previous)
								cv2.imshow("identified",copy)
								cv2.waitKey()
								cv2.destroyAllWindows()
							return self.move				
								
				# white long side castle
				if squareOne.position == "d1"  or squareTwo.position == "d1" or squareThree.position == "d1"  or squareFour.position == "d1":
					if squareOne.position == "c1"  or squareTwo.position == "c1" or squareThree.position == "c1"  or squareFour.position == "c1":
						if squareOne.position == "a1"  or squareTwo.position == "a1" or squareThree.position == "a1"  or squareFour.position == "a1":	
					
							self.move = "e1c1"
							print(self.move)
							if debug:
								squareOne.draw(copy, (255,0,0), 2)
								squareTwo.draw(copy, (255,0,0), 2)
								squareThree.draw(copy, (255,0,0),2)
								squareFour.draw(copy, (255,0,0), 2)
								cv2.imshow("previous",previous)
								cv2.imshow("identified",copy)
								cv2.waitKey()
								cv2.destroyAllWindows()
							return self.move

			# check Black short side castle
			if squareOne.position == "e8" or squareTwo.position == "e8" or squareThree.position == "e8" or  squareFour.position == "e8":
				if squareOne.position == "f8"  or squareTwo.position == "f8" or squareThree.position == "f8"  or squareFour.position == "f8":
					if squareOne.position == "g8"  or squareTwo.position == "g8" or squareThree.position == "g8"  or squareFour.position == "g8":
						if squareOne.position == "h8"  or squareTwo.position == "h8" or squareThree.position == "h8"  or squareFour.position == "h8":
							self.move = "e8g8"
							print(self.move)
							if debug:
								squareOne.draw(copy, (255,0,0), 2)
								squareTwo.draw(copy, (255,0,0), 2)
								squareThree.draw(copy, (255,0,0),2)
								squareFour.draw(copy, (255,0,0), 2)
								cv2.imshow("previous",previous)
								cv2.imshow("identified",copy)
								cv2.waitKey()
								cv2.destroyAllWindows()
							return self.move

				
				# Black long side castle
				if squareOne.position == "d8"  or squareTwo.position == "d8" or squareThree.position == "d8"  or squareFour.position == "d8":
					if squareOne.position == "c8"  or squareTwo.position == "c8" or squareThree.position == "c8"  or squareFour.position == "c8":
						if squareOne.position == "a8"  or squareTwo.position == "a8" or squareThree.position == "a8"  or squareFour.position == "a8":
							self.move = "e8c8"
							print(self.move)
							if debug:
								squareOne.draw(copy, (255,0,0), 2)
								squareTwo.draw(copy, (255,0,0), 2)
								squareThree.draw(copy, (255,0,0),2)
								squareFour.draw(copy, (255,0,0), 2)
								cv2.imshow("previous",previous)
								cv2.imshow("identified",copy)
								cv2.waitKey()
								cv2.destroyAllWindows()
							return self.move
				

		# regular move two squares change state
		squareOne = largestSquare
		squareTwo = secondLargestSquare

		if debug:
			squareOne.draw(copy, (255,0,0), 2)
			squareTwo.draw(copy, (255,0,0), 2)
			cv2.imshow("previous",previous)
			cv2.imshow("identified",copy)
			cv2.waitKey(0)
			cv2.destroyAllWindows()

		# get colors for each square from each photo
		oneCurr = squareOne.roiColor(current)
		twoCurr = squareTwo.roiColor(current)

		# calculate distance from empty square color value
		sumCurr1 = 0
		sumCurr2 = 0
		for i in range(0,3):
			sumCurr1 += (oneCurr[i] - squareOne.emptyColor[i])**2
			sumCurr2 += (twoCurr[i] - squareTwo.emptyColor[i])**2

		distCurr1 = math.sqrt(sumCurr1)
		distCurr2 = math.sqrt(sumCurr2)

		if distCurr1 < distCurr2:
			# square 1 is closer to empty color value thus empty
			squareTwo.state = squareOne.state
			squareOne.state = '.'
			# check for promotion of a pawn
			if squareTwo.state.lower() == 'p':
				if squareOne.position[1:2] == '2' and squareTwo.position[1:2] == '1':
					self.promo = True
				if squareOne.position[1:2] == '7' and squareTwo.position[1:2] == '8':
					self.promo = True

			self.move = squareOne.position + squareTwo.position

		else:
			# square 2 is currently empty
			squareOne.state = squareTwo.state
			squareTwo.state = '.'
			# check pawn promotion
			if squareOne.state.lower() == 'p':
				if squareOne.position[1:2] == '1' and squareTwo.position[1:2] == '2':
					self.promo = True
				if squareOne.position[1:2] == '8' and squareTwo.position[1:2] == '7':
					self.promo = True

					
			self.move = squareTwo.position + squareOne.position

		return self.move


In [22]:
b1 = board_Recognition()
img=cv2.imread(r'C:\Users\mayuresh\Desktop\ESE205-CVChess-master\capture1.jpg',1)
adth,s2=b1.clean_Image(img)
#print(s1)
ext=b1.initialize_mask(adth,s2)
edges,coloredges=b1.findEdges(img)
print(len(coloredges))
h1,v1=b1.findLines(edges,coloredges)
Corners=b1.findCorners(h1,v1,coloredges)
#b1.findCorners (h1, v1, colorEdges)
Squares=b1.findSquares(Corners, coloredges)

d1 = Board(Squares)
d1.draw(img)
d1.assignState()
img1=cv2.imread('C:\\Users\\mayuresh\\Desktop\\ESE205-CVChess-master\\capture2.jpg',1)
movechange=d1.determineChanges(img,img1)
print(movechange)


#import os,sys,stat



422
g1f3


In [10]:
b1 = board_Recognition()
board=b1.initialize_Board()

board.draw(img)
board.assignState()
img1=cv2.imread('C:\\Users\\mayuresh\\Desktop\\ESE205-CVChess-master\\capture2.jpg',1)
movechange=board.determineChanges(img,img1)


#os.chmod(r"C:\Users\mayuresh\Anaconda3\Lib\site-packages\stockfish",stat.S_IRWXU | stat.S_IRWXG | stat.S_IRWXO)
#os.remove()

In [23]:
import chess
import chess.uci
import numpy as np
import stockfish
from Board import Board

class ChessEng:
	'''
	This class interacts with the stockfish chess engine using the python-chess
	package. All interactions are done with the Universal Chess Interface protocol (UCI)
	Transcribes game to a txt file called Game.txt
	'''

	def __init__(self):
		'''
		Creates chessboard, local chess engine stockfish, and initiates UCI protocol
		'''

		self.engBoard = chess.Board()
		self.engine = chess.uci.popen_engine(r"C:\Users\mayuresh\Downloads\stockfish-10-win (2)\stockfish-10-win\Windows\stockfish_10_x64")
		self.engine.uci()
		print(self.engBoard)

	def updateMove(self, move):
		'''
		Updates chess board with the move made. Also checks for illegal moves
		'''

		# convert move to UCI format for engine
		uciMove = chess.Move.from_uci(move)

		# check legality
		if uciMove not in self.engBoard.legal_moves:
			return 1
		else:
			# update board
			self.engBoard.push(uciMove)
			print(self.engBoard)
			return 0


	def feedToAI(self):
		'''
		Gets the bestmove from the stockfish engine. Writes move choice to Game.txt file
		'''

		# giving the CPU the current board position
		self.engine.position(self.engBoard)
		
		# Giving the engine 2000ms to produce a move 
		response = self.engine.go(movetime=2000)
		bestMove = response[0]
		
		# update board
		self.engBoard.push(bestMove)
		
		# write move to txt file
		f = open("Game.txt", "a+")
		f.write(bestMove.uci()+ "\r\n")
		f.close()
		
		print(self.engBoard)
		return bestMove
c1 = ChessEng()
changedet=c1.updateMove(movechange)
print(c1.feedToAI())


r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . . . .
P P P P P P P P
R N B Q K B N R
r n b q k b n r
p p p p p p p p
. . . . . . . .
. . . . . . . .
. . . . . . . .
. . . . . N . .
P P P P P P P P
R N B Q K B . R
r n b q k b n r
p p p . p p p p
. . . . . . . .
. . . p . . . .
. . . . . . . .
. . . . . N . .
P P P P P P P P
R N B Q K B . R
d7d5


In [18]:
b1 = board_Recognition()
img=cv2.imread(r'C:\Users\mayuresh\Desktop\ESE205-CVChess-master\capture1.jpg',1)
adth,s2=b1.clean_Image(img)
#print(s1)
ext=b1.initialize_mask(adth,s2)
edges,coloredges=b1.findEdges(img)
print(len(coloredges))
h1,v1=b1.findLines(edges,coloredges)
Corners=b1.findCorners(h1,v1,coloredges)
#b1.findCorners (h1, v1, colorEdges)
Squares=b1.findSquares(Corners, coloredges)
d1 = Board(Squares)
d1.draw(img)
d1.assignState()
img1=cv2.imread('C:\\Users\\mayuresh\\Desktop\\ESE205-CVChess-master\\capture2.jpg',1)
movechange=d1.determineChanges(img,img1)

422


In [17]:
d1 = Board(Squares)
d1.draw(img)
d1.assignState()
img1=cv2.imread('C:\\Users\\mayuresh\\Desktop\\ESE205-CVChess-master\\capture2.jpg',1)
movechange=d1.determineChanges(img,img1)